In [1]:
import py7zr
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
# --- Extract CSV from 7z ---
DATA_PATH = r"D:\Assessment - SPTD Specialist AI_Data\SectionB_taxi trips_201501 to 201509.7z"
OUTPUT_PATH = r"D:\Assessment - SPTD Specialist AI_Data\SectionB_taxi trips_201501 to 201509"

os.makedirs(OUTPUT_PATH, exist_ok=True)

with py7zr.SevenZipFile(DATA_PATH, mode='r') as archive:
    csv_files = [f for f in archive.getnames() if f.lower().endswith("csv")]
    
    for f in csv_files:
        output_file = os.path.join(OUTPUT_PATH, f)
        if os.path.exists(output_file):
            os.remove(output_file)

    archive.extract(targets=csv_files, path=OUTPUT_PATH)

print(f"Extracted {len(csv_files)} CSV files to {OUTPUT_PATH}")

Extracted 9 CSV files to D:\Assessment - SPTD Specialist AI_Data\SectionB_taxi trips_201501 to 201509


In [ ]:
# --- Headers ---
def clean_headers(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"\(.*?\)", "", regex=True)
    )
    return df

# --- Travel Date & Time ---
def clean_datetime(df):
    date_split = df["travel_date"].str.split(" TO ", expand=True)
    time_split = df["travel_time"].str.split(" TO ", expand=True)

    df["start_datetime"] = pd.to_datetime(
        date_split[0] + " " + time_split[0],
        dayfirst=True,
        errors="coerce"
    )

    df["end_datetime"] = pd.to_datetime(
        date_split[1] + " " + time_split[1],
        dayfirst=True,
        errors="coerce"
    )

    df["trip_duration_min"] = (
        df["end_datetime"] - df["start_datetime"]
    ).dt.total_seconds() / 60
    
    return df

# --- Handle Repeated Cols (card_no, pickup_postal, travel_date, taxi_fare) ---
def clean_repetition(df, base_col, drop_extras=True):
    candidates = [c for c in df.columns if c.startswith(base_col)]

    if not candidates:
        raise ValueError(f"No columns found starting with '{base_col}'")
    
    df[base_col] = df[candidates].bfill(axis=1).iloc[:,0]

    if drop_extras:
        extras = [c for c in candidates if c != base_col]
        df = df.drop(columns=extras)

    return df

In [25]:
def load_and_process_folder(folder_path):
    all_files = Path(folder_path).glob("TripReport_*.csv")
    dfs = []

    for file in all_files:
        print(f"Processing file: {os.path.basename(file)}...")
        df = pd.read_csv(file)     
        print(f"File Size: {data.shape}")        

        df = df.drop(columns=[
            'Pickup_X', 'Pickup_Y',
            'Destination_X', 'Destination_Y'          
        ])        

        df = df.replace(r"^\s*(nil|null)?\s*$", pd.NA, regex=True)      
        df = clean_headers(df)                      
       
        # base_cols = ["cardno", "pickup_postal", "travel_date", "taxi_fare"]
        # for base_col in base_cols:
        #     df = clean_repetition(df, base_col)

        # df = clean_datetime(df)

        # data["total_fare"] = data["taxi_fare"] + data["admin"]

        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

data = load_and_process_folder(r"D:\Assessment - SPTD Specialist AI_Data\SectionB_taxi trips_201501 to 201509")

print(data.shape)
print(data.columns)
pd.set_option("display.max_colwidth", None)
print(data.to_string(index=False))

Processing file: TripReport_201501.csv...
File Size: (1094, 15)
Processing file: TripReport_201502.csv...
File Size: (1094, 15)
Processing file: TripReport_201503.csv...
File Size: (1094, 15)
Processing file: TripReport_201504.csv...
File Size: (1094, 15)
Processing file: TripReport_201505.csv...
File Size: (1094, 15)
Processing file: TripReport_201506.csv...
File Size: (1094, 15)
Processing file: TripReport_201507.csv...
File Size: (1094, 15)
Processing file: TripReport_201508.csv...
File Size: (1094, 15)
Processing file: TripReport_201509.csv...
File Size: (1094, 15)
(1094, 15)
Index(['cardno', 'travel_date', 'travel_time', 'taxi_no', 'pickup_postal',
       'pickup_latitude', 'pickup_longtitude', 'destination_postal',
       'destination_latitude', 'destination_longtitude', 'taxi_fare', 'admin',
       'distance_run', 'division_code', 'card_no'],
      dtype='object')
           cardno               travel_date          travel_time  taxi_no pickup_postal  pickup_latitude  pickup_lon